In [169]:
import pandas as pd
import numpy as np
import os
import re

# Preprocessing Header

In [170]:
df = pd.read_csv("../Raw Data/extracted_header.csv")

In [171]:
df.head(n=3)

,sender_email,reply_to_email,return_path,inline_image_count,Label
0,rssfeeds@example.com,NaN,rssfeeds@example.com,0,Ham
1,thompson@shelob.ce.ttu.edu,exmh-users@example.com,exmh-users-admin@example.com,0,Ham
2,beberg@mithral.com,NaN,fork-admin@xent.com,0,Ham


In [172]:
df.isna().sum()

sender_email           130
reply_to_email        9312
return_path            340
inline_image_count       0
Label                    0
dtype: int64

In [173]:
df.shape

(12827, 5)

In [174]:
df.Label.value_counts() / len(df) * 100

Spam    81.211507
Ham     18.788493
Name: Label, dtype: float64

In [175]:
df.groupby("Label")["inline_image_count"].agg(["mean","median"])

,mean,median
Label,,
Ham,0.000000,0
Spam,0.607085,0


In [176]:
df[df.Label == "Ham"]["inline_image_count"].unique() # All Ham emails don't have a single inline image

array([0])

In [177]:
a=df[df.Label == "Spam"]["inline_image_count"]

In [178]:
(a == 0).sum() / len(a) * 100 # 86% of Spam Emails don't have a single inline image

85.79245464145147

In [179]:
df["has_inline_image"] = df["inline_image_count"] >= 1
df.drop("inline_image_count",axis=1,inplace=True)

In [180]:
df[df.Label == "Ham"][["sender_email","reply_to_email","return_path"]].sample(10)

,sender_email,reply_to_email,return_path
513,signa@birch.net,NaN,fork-admin@xent.com
2368,rick@linuxmafia.com,NaN,ilug-admin@linux.ie
323,bitbitch@magnesium.net,bitbitch@magnesium.net,fork-admin@xent.com
1129,matthias@egwn.net,rpm-zzzlist@freshrpms.net,rpm-zzzlist-admin@freshrpms.net
828,eugen@leitl.org,NaN,fork-admin@xent.com
1251,tomwhore@slack.net,NaN,fork-admin@xent.com
2327,rssfeeds@example.com,NaN,rssfeeds@example.com
809,rssfeeds@example.com,NaN,rssfeeds@example.com
95,bitbitch@magnesium.net,bitbitch@magnesium.net,fork-admin@xent.com
338,rssfeeds@example.com,NaN,rssfeeds@example.com


In [181]:
df[df.Label == "Spam"][["sender_email","reply_to_email","return_path"]].sample(10)

,sender_email,reply_to_email,return_path
6783,rptksyn@smapxsmap.com,NaN,rptksyn@smapxsmap.com
8663,adam@1hotelsvietnam.com,NaN,adam@1hotelsvietnam.com
4956,pahpop07vhrl@inetworld.net,pahpop07vhrl@inetworld.net,pahpop07vhrl@inetworld.net
8671,xkx18dovyu@lynchburg.net,xkx18dovyu@lynchburg.net,xkx18dovyu@lynchburg.net
5794,kxsice@elks1605.org,NaN,kxsice@elks1605.org
10409,h61wjn@yahoo.com,h61wjn@yahoo.com,h61wjn@yahoo.com
8127,consciencem7@tele2.fr,NaN,consciencem7@tele2.fr
5197,Santos@portalen.no,NaN,Santos@portalen.no
2415,safety33o@l13.newnamedns.com,safety33o@l13.newnamedns.com,safety33o@l13.newnamedns.com
3080,mrs_stellauba@terra.com.mx,NaN,mrs_stellauba@terra.com.mx


In [182]:
df.isna().sum() / len(df) * 100

sender_email         1.013487
reply_to_email      72.596866
return_path          2.650659
Label                0.000000
has_inline_image     0.000000
dtype: float64

In [183]:
df["no_reply_to_email"] = df["reply_to_email"].isna() # true -> does not have
df.groupby("Label")["no_reply_to_email"].agg(["mean"])*100

,mean
Label,
Ham,74.439834
Spam,72.170491


In [184]:
df["no_return_path"] = df["return_path"].isna() # true -> does not have
df.groupby("Label")["no_return_path"].agg(["mean"])*100

# 3.26% of spam don't have return path

,mean
Label,
Ham,0.000000
Spam,3.263896


In [185]:
df[df["no_return_path"] == True]

,sender_email,reply_to_email,return_path,Label,has_inline_image,no_reply_to_email,no_return_path
2484,rkkss@redseven.de,NaN,NaN,Spam,False,True,True
2541,rkkss@redseven.de,NaN,NaN,Spam,False,True,True
2571,emailrewardz@emailrewardz.email-publisher.com,perf-remove.3565.64698.13893713.0.0.4@boing.to...,NaN,Spam,False,False,True
2693,meg34807147238s03@isppan.waw.pl,meg34807147238s03@isppan.waw.pl,NaN,Spam,False,False,True
2745,meg34807147238s03@isppan.waw.pl,meg34807147238s03@isppan.waw.pl,NaN,Spam,False,False,True
...,...,...,...,...,...,...,...
12625,NaN,NaN,NaN,Spam,False,True,True
12639,YCPZHUNZN@marchmail.com,YCPZHUNZN@marchmail.com,NaN,Spam,False,False,True
12648,keh-ming@a-vip.com,NaN,NaN,Spam,False,True,True
12747,505jeff@acadia.net,NaN,NaN,Spam,False,True,True


In [186]:
a=df[(df["no_reply_to_email"]==True) & (df["no_return_path"] == True)]
print(a["Label"].value_counts() / len(a) * 100)
print("Overall Distribution of class:")
print(df.Label.value_counts() / len(df) * 100)
print("% of such records: " + str(len(a)/len(df)*100))
print(a.sample(7)[["Label","no_reply_to_email","no_return_path"]])

Spam    100.0
Name: Label, dtype: float64
Overall Distribution of class:
Spam    81.211507
Ham     18.788493
Name: Label, dtype: float64
% of such records: 2.3388165588212364
      Label  no_reply_to_email  no_return_path
10699  Spam               True            True
3115   Spam               True            True
3310   Spam               True            True
9299   Spam               True            True
6567   Spam               True            True
3773   Spam               True            True
12428  Spam               True            True


In [187]:
a=df[(df["no_reply_to_email"]==False) & (df["no_return_path"] == False)]
print(a["Label"].value_counts() / len(a) * 100)
print("Overall Distribution of class:")
print(df.Label.value_counts() / len(df) * 100)
print("% of such records: " + str(len(a)/len(df)*100))
print(a.sample(7)[["Label","no_reply_to_email","no_return_path"]])

Spam    82.273381
Ham     17.726619
Name: Label, dtype: float64
Overall Distribution of class:
Spam    81.211507
Ham     18.788493
Name: Label, dtype: float64
% of such records: 27.09129180634599
      Label  no_reply_to_email  no_return_path
5646   Spam              False           False
3303   Spam              False           False
4855   Spam              False           False
12594  Spam              False           False
1213    Ham              False           False
6344   Spam              False           False
5696   Spam              False           False


In [188]:
a=df[(df["no_reply_to_email"]==True) & (df["no_return_path"] == False)]
print(a["Label"].value_counts() / len(a) * 100)
print("Overall Distribution of class:")
print(df.Label.value_counts() / len(df) * 100)
print("% of such records: " + str(len(a)/len(df)*100))
print(a.sample(7)[["Label","no_reply_to_email","no_return_path"]])

Spam    80.093209
Ham     19.906791
Name: Label, dtype: float64
Overall Distribution of class:
Spam    81.211507
Ham     18.788493
Name: Label, dtype: float64
% of such records: 70.25804942698994
      Label  no_reply_to_email  no_return_path
8341   Spam               True           False
11450  Spam               True           False
5105   Spam               True           False
11517  Spam               True           False
6134   Spam               True           False
8443   Spam               True           False
11978  Spam               True           False


In [189]:
a=df[(df["no_reply_to_email"]==False) & (df["no_return_path"] == True)]
print(a["Label"].value_counts() / len(a) * 100)
print("Overall Distribution of class:")
print(df.Label.value_counts() / len(df) * 100)
print("% of such records: " + str(len(a)/len(df)*100))
print(a.sample(7)[["Label","no_reply_to_email","no_return_path"]])

Spam    100.0
Name: Label, dtype: float64
Overall Distribution of class:
Spam    81.211507
Ham     18.788493
Name: Label, dtype: float64
% of such records: 0.3118422078428315
      Label  no_reply_to_email  no_return_path
11532  Spam              False            True
11826  Spam              False            True
9525   Spam              False            True
11694  Spam              False            True
5080   Spam              False            True
6453   Spam              False            True
8690   Spam              False            True


In [190]:
df["no_reply_no_return"] = (df["no_reply_to_email"] == True) & (df["no_return_path"] == True)
df["yes_reply_no_return"] = (df["no_reply_to_email"] == False) & (df["no_return_path"] == True)
print(df.columns)

Index(['sender_email', 'reply_to_email', 'return_path', 'Label',
       'has_inline_image', 'no_reply_to_email', 'no_return_path',
       'no_reply_no_return', 'yes_reply_no_return'],
      dtype='object')


In [191]:
df.drop(["no_reply_to_email","reply_to_email","return_path"],axis=1,inplace=True)
df.head()

,sender_email,Label,has_inline_image,no_return_path,no_reply_no_return,yes_reply_no_return
0,rssfeeds@example.com,Ham,False,False,False,False
1,thompson@shelob.ce.ttu.edu,Ham,False,False,False,False
2,beberg@mithral.com,Ham,False,False,False,False
3,rssfeeds@example.com,Ham,False,False,False,False
4,garym@canada.com,Ham,False,False,False,False


In [192]:
df.isna().sum()

sender_email           130
Label                    0
has_inline_image         0
no_return_path           0
no_reply_no_return       0
yes_reply_no_return      0
dtype: int64

In [193]:
df = df.dropna()

In [194]:
tld_pattern_compiled = re.compile(r"(@.+\.)(\w+)")
def get_tld(email):
    return tld_pattern_compiled.search(email).groups()[1]
    
df["top_level_domain"] = df["sender_email"].apply(get_tld)

In [195]:
df["top_level_domain"].value_counts()

com     6873
net     2108
org      424
de       311
uk       302
        ... 
ms         1
ac         1
InFo       1
uy         1
118        1
Name: top_level_domain, Length: 150, dtype: int64

In [196]:
df.head()

,sender_email,Label,has_inline_image,no_return_path,no_reply_no_return,yes_reply_no_return,top_level_domain
0,rssfeeds@example.com,Ham,False,False,False,False,com
1,thompson@shelob.ce.ttu.edu,Ham,False,False,False,False,edu
2,beberg@mithral.com,Ham,False,False,False,False,com
3,rssfeeds@example.com,Ham,False,False,False,False,com
4,garym@canada.com,Ham,False,False,False,False,com


In [222]:
account_pattern_compiled = re.compile(r"(.+@)")
def get_account(email):
    return account_pattern_compiled.search(email).groups()[0][:-1]
    
df["account_name"] = df["sender_email"].apply(get_account)

In [223]:
domain_pattern_compiled = re.compile(r"(@.+)(\.\w+)")
def get_domain(email):
    return domain_pattern_compiled.search(email).groups()[0][1:]
    
df["domain"] = df["sender_email"].apply(get_domain)

In [224]:
df[["sender_email","account_name","top_level_domain","domain"]].sample(n=7)

,sender_email,account_name,top_level_domain,domain
9878,YPIHSWLBA@moose-mail.com,YPIHSWLBA,com,moose-mail
11963,george_williams3@virgilio.it,george_williams3,it,virgilio
7447,envet@yahoo.com,envet,com,yahoo
8660,horrendous@ozpod.com,horrendous,com,ozpod
5822,candlesrich@libero.it,candlesrich,it,libero
10972,ceks@reflex.at,ceks,at,reflex
4533,sasiporn_joom@hotmail.com,sasiporn_joom,com,hotmail
